# 🔧 环境配置和检查

#### 概述

本教程需要特定的环境配置以确保最佳学习体验。以下配置将帮助你：

- 使用统一的conda环境：激活统一的学习环境
- 通过国内镜像源快速安装依赖：配置pip使用清华镜像源
- 加速模型下载：设置HuggingFace镜像代理
- 检查系统配置：检查硬件和软件配置

#### 配置

- **所需环境及其依赖已经部署好**
- 在`Notebook`右上角选择`jupyter内核`为`python(agent101)`，即可执行


In [1]:
%%script bash

# 1. 激活 conda 环境 (仅对当前单元格有效)
eval "$(conda shell.bash hook)"
conda activate agent101

echo "========================================="
echo "== Conda 环境检查报告 (仅针对当前 Bash 子进程) =="
echo "========================================="

# 2. 检查当前激活的环境
CURRENT_ENV_NAME=$(basename $CONDA_PREFIX)

if [ "$CURRENT_ENV_NAME" = "agent101" ]; then
    echo "✅ 当前单元格已成功激活到 agent101 环境。"
    echo "✅ 正在使用的环境路径: $CONDA_PREFIX"
    echo ""
    echo "💡 提示: 后续的Python单元格将使用Notebook当前选择的Jupyter内核。"
    echo "   如果需要后续单元格也使用此环境，请执行以下操作:"
    echo "   1. 检查 Notebook 右上角是否已选择 'python(agent101)'。"
else
    echo "❌ 激活失败或环境名称不匹配。当前环境: $CURRENT_ENV_NAME"
    echo ""
    echo "⚠️ 严重提示: 建议将 Notebook 的 Jupyter **内核 (Kernel)** 切换为 'python(agent101)'。"
    echo "   (通常位于 Notebook 右上角或 '内核' 菜单中)"
    echo ""
    echo "📚 备用方法 (不推荐): 如果无法切换内核，则必须在**每个**代码单元格的头部重复以下命令:"
    echo ""
    echo "%%script bash"
    echo "# 必须在每个单元格都执行"
    echo "eval \"\$(conda shell.bash hook)\""
    echo "conda activate agent101"
fi

echo "========================================="


== Conda 环境检查报告 (仅针对当前 Bash 子进��) ==
✅ 当前单元格已成功激活到 agent101 环境。
✅ 正在使用的环境路径: /root/miniconda3/envs/agent101

💡 提示: 后续的Python单元格将使用Notebook当前选择的Jupyter内核。
   如果需���后续单元格也使用此环境，请执行以下操作:
   1. 检查 Notebook 右上角是否��选择 'python(agent101)'。


In [2]:
# 2. 设置pip 为清华源
%pip config list -v set global.index-url https://pypi.tuna.tsinghua.edu.cn/simple
%pip config list -v


Writing to /root/.config/pip/pip.conf
Note: you may need to restart the kernel to use updated packages.
ERROR: Need an action (debug, edit, get, list, set, unset) to perform.
Note: you may need to restart the kernel to use updated packages.


In [3]:
# 3. 设置HuggingFace代理
%env HF_ENDPOINT=https://hf-mirror.com
# 验证：使用shell命令检查
!echo $HF_ENDPOINT


env: HF_ENDPOINT=https://hf-mirror.com
https://hf-mirror.com


In [4]:
# 🔍 环境信息检查脚本
#
# 本脚本的作用：
# 1. 安装 pandas 库用于数据表格展示
# 2. 检查系统的各项配置信息
# 3. 生成详细的环境报告表格
#
# 对于初学者来说，这个步骤帮助你：
# - 了解当前运行环境的硬件配置
# - 确认是否满足模型运行的最低要求
# - 学习如何通过代码获取系统信息

# 安装 pandas 库 - 用于创建和展示数据表格
# pandas 是 Python 中最流行的数据处理和分析库
%pip install pandas==2.2.2 tabulate==0.9.0

import platform # 导入 platform 模块以获取系统信息
import os # 导入 os 模块以与操作系统交互
import subprocess # 导入 subprocess 模块以运行外部命令
import pandas as pd # 导入 pandas 模块，通常用于数据处理，这里用于创建表格
import shutil # 导入 shutil 模块以获取磁盘空间信息

# 获取 CPU 信息的函数，包括核心数量
def get_cpu_info():
    cpu_info = "" # 初始化 CPU 信息字符串
    physical_cores = "N/A"
    logical_cores = "N/A"

    if platform.system() == "Windows": # 如果是 Windows 系统
        cpu_info = platform.processor() # 使用 platform.processor() 获取 CPU 信息
        try:
            # 获取 Windows 上的核心数量 (需要 WMI)
            import wmi
            c = wmi.WMI()
            for proc in c.Win32_Processor():
                physical_cores = proc.NumberOfCores
                logical_cores = proc.NumberOfLogicalProcessors
        except:
            pass # 如果 WMI 不可用，忽略错误

    elif platform.system() == "Darwin": # 如果是 macOS 系统
        # 在 macOS 上使用 sysctl 命令获取 CPU 信息和核心数量
        os.environ['PATH'] = os.environ['PATH'] + os.pathsep + '/usr/sbin' # 更新 PATH 环境变量
        try:
            process_brand = subprocess.Popen(['sysctl', "machdep.cpu.brand_string"], stdout=subprocess.PIPE, stderr=subprocess.PIPE)
            stdout_brand, stderr_brand = process_brand.communicate()
            cpu_info = stdout_brand.decode().split(': ')[1].strip() if stdout_brand else "Could not retrieve CPU info"

            process_physical = subprocess.Popen(['sysctl', "hw.physicalcpu"], stdout=subprocess.PIPE, stderr=subprocess.PIPE)
            stdout_physical, stderr_physical = process_physical.communicate()
            physical_cores = stdout_physical.decode().split(': ')[1].strip() if stdout_physical else "N/A"

            process_logical = subprocess.Popen(['sysctl', "hw.logicalcpu"], stdout=subprocess.PIPE, stderr=subprocess.PIPE)
            stdout_logical, stderr_logical = process_logical.communicate()
            logical_cores = stdout_logical.decode().split(': ')[1].strip() if stdout_logical else "N/A"

        except:
            cpu_info = "Could not retrieve CPU info"
            physical_cores = "N/A"
            logical_cores = "N/A"

    else:  # Linux 系统
        try:
            # 在 Linux 上读取 /proc/cpuinfo 文件获取 CPU 信息和核心数量
            with open('/proc/cpuinfo') as f:
                physical_cores_count = 0
                logical_cores_count = 0
                cpu_info_lines = []
                for line in f:
                    if line.startswith('model name'): # 查找以 'model name'开头的行
                        if not cpu_info: # 只获取第一个 model name
                            cpu_info = line.split(': ')[1].strip()
                    elif line.startswith('cpu cores'): # 查找以 'cpu cores' 开头的行
                        physical_cores_count = int(line.split(': ')[1].strip())
                    elif line.startswith('processor'): # 查找以 'processor' 开头的行
                        logical_cores_count += 1
                physical_cores = str(physical_cores_count) if physical_cores_count > 0 else "N/A"
                logical_cores = str(logical_cores_count) if logical_cores_count > 0 else "N/A"
                if not cpu_info:
                     cpu_info = "Could not retrieve CPU info"

        except:
            cpu_info = "Could not retrieve CPU info"
            physical_cores = "N/A"
            logical_cores = "N/A"

    return f"{cpu_info} ({physical_cores} physical cores, {logical_cores} logical cores)" # 返回 CPU 信息和核心数量


# 获取内存信息的函数
def get_memory_info():
    mem_info = "" # 初始化内存信息字符串
    if platform.system() == "Windows":
        # 在 Windows 上不容易通过标准库获取，需要外部库或 PowerShell
        mem_info = "Requires external tools on Windows" # 设置提示信息
    elif platform.system() == "Darwin": # 如果是 macOS 系统
        # 在 macOS 上使用 sysctl 命令获取内存大小
        process = subprocess.Popen(['sysctl', "hw.memsize"], stdout=subprocess.PIPE, stderr=subprocess.PIPE) # 运行 sysctl 命令
        stdout, stderr = process.communicate() # 获取标准输出和标准错误
        mem_bytes = int(stdout.decode().split(': ')[1].strip()) # 解析输出，获取内存大小（字节）
        mem_gb = mem_bytes / (1024**3) # 转换为 GB
        mem_info = f"{mem_gb:.2f} GB" # 格式化输出
    else:  # Linux 系统
        try:
            # 在 Linux 上读取 /proc/meminfo 文件获取内存信息
            with open('/proc/meminfo') as f:
                total_mem_kb = 0
                available_mem_kb = 0
                for line in f:
                    if line.startswith('MemTotal'): # 查找以 'MemTotal' 开头的行
                        total_mem_kb = int(line.split(':')[1].strip().split()[0]) # 解析行，获取总内存（KB）
                    elif line.startswith('MemAvailable'): # 查找以 'MemAvailable' 开头的行
                         available_mem_kb = int(line.split(':')[1].strip().split()[0]) # 解析行，获取可用内存（KB）

                if total_mem_kb > 0:
                    total_mem_gb = total_mem_kb / (1024**2) # 转换为 GB
                    mem_info = f"{total_mem_gb:.2f} GB" # 格式化输出总内存
                    if available_mem_kb > 0:
                        available_mem_gb = available_mem_kb / (1024**2)
                        mem_info += f" (Available: {available_mem_gb:.2f} GB)" # 添加可用内存信息
                else:
                     mem_info = "Could not retrieve memory info" # 如果读取文件出错，设置错误信息

        except:
            mem_info = "Could not retrieve memory info" # 如果读取文件出错，设置错误信息
    return mem_info # 返回内存信息

# 获取 GPU 信息的函数，包括显存
def get_gpu_info():
    try:
        # 尝试使用 nvidia-smi 获取 NVIDIA GPU 信息和显存
        result = subprocess.run(['nvidia-smi', '--query-gpu=name,memory.total', '--format=csv,noheader'], capture_output=True, text=True)
        if result.returncode == 0: # 如果命令成功执行
            gpu_lines = result.stdout.strip().split('\n') # 解析输出，获取 GPU 名称和显存
            gpu_info_list = []
            for line in gpu_lines:
                name, memory = line.split(', ')
                gpu_info_list.append(f"{name} ({memory})") # 格式化 GPU 信息
            return ", ".join(gpu_info_list) if gpu_info_list else "NVIDIA GPU found, but info not listed" # 返回 GPU 信息或提示信息
        else:
             # 尝试使用 lshw 获取其他 GPU 信息 (需要安装 lshw)
            try:
                result_lshw = subprocess.run(['lshw', '-C', 'display'], capture_output=True, text=True)
                if result_lshw.returncode == 0: # 如果命令成功执行
                     # 简单解析输出中的 product 名称和显存
                    gpu_info_lines = []
                    current_gpu = {}
                    for line in result_lshw.stdout.splitlines():
                        if 'product:' in line:
                             if current_gpu:
                                 gpu_info_lines.append(f"{current_gpu.get('product', 'GPU')} ({current_gpu.get('memory', 'N/A')})")
                             current_gpu = {'product': line.split('product:')[1].strip()}
                        elif 'size:' in line and 'memory' in line:
                             current_gpu['memory'] = line.split('size:')[1].strip()

                    if current_gpu: # 添加最后一个 GPU 的信息
                        gpu_info_lines.append(f"{current_gpu.get('product', 'GPU')} ({current_gpu.get('memory', 'N/A')})")

                    return ", ".join(gpu_info_lines) if gpu_info_lines else "GPU found (via lshw), but info not parsed" # 如果找到 GPU 但信息无法解析，设置提示信息
                else:
                    return "No GPU found (checked nvidia-smi and lshw)" # 如果两个命令都找不到 GPU，设置提示信息
            except FileNotFoundError:
                 return "No GPU found (checked nvidia-smi, lshw not found)" # 如果找不到 lshw 命令，设置提示信息
    except FileNotFoundError:
        return "No GPU found (nvidia-smi not found)" # 如果找不到 nvidia-smi 命令，设置提示信息


# 获取 CUDA 版本的函数
def get_cuda_version():
    try:
        # 尝试使用 nvcc --version 获取 CUDA 版本
        result = subprocess.run(['nvcc', '--version'], capture_output=True, text=True)
        if result.returncode == 0: # 如果命令成功执行
            for line in result.stdout.splitlines():
                if 'release' in line: # 查找包含 'release' 的行
                    return line.split('release ')[1].split(',')[0] # 解析行，提取版本号
        return "CUDA not found or version not parsed" # 如果找不到 CUDA 或版本无法解析，设置提示信息
    except FileNotFoundError:
        return "CUDA not found" # 如果找不到 nvcc 命令，设置提示信息

# 获取 Python 版本的函数
def get_python_version():
    return platform.python_version() # 获取 Python 版本

# 获取 Conda 版本的函数
def get_conda_version():
    try:
        # 尝试使用 conda --version 获取 Conda 版本
        result = subprocess.run(['conda', '--version'], capture_output=True, text=True)
        if result.returncode == 0: # 如果命令成功执行
            return result.stdout.strip() # 返回 Conda 版本
        return "Conda not found or version not parsed" # 如果找不到 Conda 或版本无法解析，设置提示信息
    except FileNotFoundError:
        return "Conda not found" # 如果找不到 conda 命令，设置提示信息

# 获取物理磁盘空间信息的函数
def get_disk_space():
    try:
        total, used, free = shutil.disk_usage("/") # 获取根目录的磁盘使用情况
        total_gb = total / (1024**3) # 转换为 GB
        used_gb = used / (1024**3) # 转换为 GB
        free_gb = free / (1024**3) # 转换为 GB
        return f"Total: {total_gb:.2f} GB, Used: {used_gb:.2f} GB, Free: {free_gb:.2f} GB" # 格式化输出
    except Exception as e:
        return f"Could not retrieve disk info: {e}" # 如果获取信息出错，设置错误信息

# 获取环境信息
os_name = platform.system() # 获取操作系统名称
os_version = platform.release() # 获取操作系统版本
if os_name == "Linux":
    try:
        # 在 Linux 上尝试获取发行版和版本
        lsb_info = subprocess.run(['lsb_release', '-a'], capture_output=True, text=True)
        if lsb_info.returncode == 0: # 如果命令成功执行
            for line in lsb_info.stdout.splitlines():
                if 'Description:' in line: # 查找包含 'Description:' 的行
                    os_version = line.split('Description:')[1].strip() # 提取描述信息作为版本
                    break # 找到后退出循环
                elif 'Release:' in line: # 查找包含 'Release:' 的行
                     os_version = line.split('Release:')[1].strip() # 提取版本号
                     # 尝试获取 codename
                     try:
                         codename_info = subprocess.run(['lsb_release', '-c'], capture_output=True, text=True)
                         if codename_info.returncode == 0:
                             os_version += f" ({codename_info.stdout.split(':')[1].strip()})" # 将 codename 添加到版本信息中
                     except:
                         pass # 如果获取 codename 失败则忽略

    except FileNotFoundError:
        pass # lsb_release 可能未安装，忽略错误

full_os_info = f"{os_name} {os_version}" # 组合完整的操作系统信息
cpu_info = get_cpu_info() # 调用函数获取 CPU 信息和核心数量
memory_info = get_memory_info() # 调用函数获取内存信息
gpu_info = get_gpu_info() # 调用函数获取 GPU 信息和显存
cuda_version = get_cuda_version() # 调用函数获取 CUDA 版本
python_version = get_python_version() # 调用函数获取 Python 版本
conda_version = get_conda_version() # 调用函数获取 Conda 版本
disk_info = get_disk_space() # 调用函数获取物理磁盘空间信息


# 创建用于存储数据的字典
env_data = {
    "项目": [ # 项目名称列表
        "操作系统",
        "CPU 信息",
        "内存信息",
        "GPU 信息",
        "CUDA 信息",
        "Python 版本",
        "Conda 版本",
        "物理磁盘空间" # 添加物理磁盘空间
    ],
    "信息": [ # 对应的信息列表
        full_os_info,
        cpu_info,
        memory_info,
        gpu_info,
        cuda_version,
        python_version,
        conda_version,
        disk_info # 添加物理磁盘空间信息
    ]
}

# 创建一个 pandas DataFrame
df = pd.DataFrame(env_data)

# 打印表格
print("### 环境信息") # 打印标题
print(df.to_markdown(index=False)) # 将 DataFrame 转换为 Markdown 格式并打印，不包含索引


Looking in indexes: https://pypi.tuna.tsinghua.edu.cn/simple
Note: you may need to restart the kernel to use updated packages.
### 环境信息
| 项目         | 信息                                                                               |
|:-------------|:-----------------------------------------------------------------------------------|
| 操作系统     | Linux Ubuntu 22.04.4 LTS                                                           |
| CPU 信息     | 11th Gen Intel(R) Core(TM) i5-1135G7 @ 2.40GHz (1 physical cores, 2 logical cores) |
| 内存信息     | 3.78 GB (Available: 1.21 GB)                                                       |
| GPU 信息     | No GPU found (nvidia-smi not found)                                                |
| CUDA 信息    | CUDA not found                                                                     |
| Python 版本  | 3.10.18                                                                            |
| Conda 版本   | conda 24.4.0                                         

# 工具使用


## 设置

运行以下设置单元格来加载你的 API 密钥并建立 `get_completion` 辅助函数。

In [5]:
# 🔧 OpenAI环境自动配置
# 此设置会自动从环境变量或IPython存储中加载配置

# 安装OpenAI库
%pip install openai==1.107.0

# 导入Python内置的正则表达式库
import re

# 🚀 使用统一配置管理系统
from config import setup_notebook_environment, print_config_info

# 自动设置OpenAI客户端和get_completion函数
# 优先级：环境变量 > IPython存储 > 默认值
try:
    client, get_completion = setup_notebook_environment()
    print("✅ 使用统一配置管理成功！")
except Exception as e:
    print(f"❌ 统一配置失败，回退到传统方式: {e}")
    
    # 回退到传统的配置方式
    import openai
    
    # 从IPython存储中检索API_KEY和MODEL_NAME变量
    %store -r API_KEY
    %store -r MODEL_NAME

    # 如果没有设置MODEL_NAME，使用默认值
    try:
        MODEL_NAME
    except NameError:
        MODEL_NAME = "gpt-4o"  # 默认使用gpt-4o模型

    # 创建OpenAI客户端
    client = openai.OpenAI(api_key=API_KEY)

    def get_completion(prompt: str, system_prompt=""):
        """
        获取GPT的完成响应
        
        参数:
            prompt (str): 用户提示
            system_prompt (str): 系统提示（可选）
        
        返回:
            str: GPT的响应文本
        """
        # 构建消息列表
        messages = []
        
        # 如果有系统提示，添加系统消息
        if system_prompt:
            messages.append({"role": "system", "content": system_prompt})
        
        # 添加用户消息
        messages.append({"role": "user", "content": prompt})
        
        # 调用OpenAI API
        response = client.chat.completions.create(
            model=MODEL_NAME,              # 模型名称 (gpt-4o 或 deepseek-r1)
            messages=messages,             # 消息列表
            max_completion_tokens=2000,    # 最大token数
            temperature=0.0               # 温度参数，0表示更确定性
        )
        return response.choices[0].message.content
    
    print("⚠️  使用传统配置方式，建议配置环境变量以获得更好体验")

Looking in indexes: https://pypi.tuna.tsinghua.edu.cn/simple
Note: you may need to restart the kernel to use updated packages.
✅ OpenAI环境设置完成!
🔧 OpenAI API 配置信息:
  📡 配置来源: 环境变量 + 自定义API地址: https://api.apiyi.com/v1
  🤖 模型: deepseek-chat
  🌐 API地址: https://api.apiyi.com/v1
  🔑 API密钥: sk-AGnnT...A14f

✅ 使用统一配置管理成功！


---

## 课程
工具使用（也叫函数调用）看起来复杂，但其实很简单！它就是将工具执行结果插入到对话中，让AI能调用外部功能。

**核心原理：**
- 普通对话：我们直接给AI发送文本
- 工具使用：我们让AI告诉我们要调用什么工具，然后把工具结果返回给AI

**工作流程：**
1. AI 说明要调用的工具名称和参数
2. 我们的程序执行这个工具
3. 把执行结果返回给AI继续对话

## 为什么需要工具使用？

工具使用让AI能力大幅扩展，可以处理复杂的实际任务：

**常见工具类型：**
- 📊 数据库查询：执行SQL语句，获取数据
- 🌐 网络请求：调用API，获取实时信息  
- 📁 文件操作：读写文件，处理文档
- 🧮 计算工具：数学运算，数据分析
- 📨 消息发送：邮件、短信、通知

## 实现工具使用的两个关键要素

**1. 系统提示词**
- 告诉AI有哪些工具可用
- 说明每个工具的用途和参数
- 定义工具调用的格式

**2. 控制逻辑**  
- 解析AI的工具调用请求
- 执行对应的工具函数
- 将结果返回给AI继续对话

### 工具使用路线图

*本课程教授我们当前的工具使用格式。但是，我们将在不久的将来更新和改进工具使用功能，包括：*
* *更简化的函数定义和调用格式*
* *更强健的错误处理和边缘情况覆盖*
* *与我们 API 其余部分的更紧密集成*
* *更好的可靠性和性能，特别是对于更复杂的工具使用任务*

## 实战示例：SQL数据库查询工具

下面我们通过一个具体的SQL查询例子来学习工具使用。

### 1. 系统提示词设计

系统提示词需要包含：
- 工具使用的基本说明
- 具体工具的详细描述  
- 调用格式要求

**重要：** 使用 `<function_calls>` 标签是GPT专门训练的格式，建议严格遵循。

In [6]:
# 构建系统提示词 - 优化版本
system_prompt = """
你是一个智能数据分析助手，必须使用提供的工具来查询数据库。

## 可用工具

### sql_query
功能：执行SQL查询语句，获取数据库数据
参数：
- query (字符串): 要执行的SQL查询语句

## 重要规则

1. 当用户询问数据相关问题时，你必须使用sql_query工具查询
2. 不要直接回答，必须先调用工具获取数据
3. 工具调用后立即停止生成，等待工具结果

## 工具调用格式

当你需要查询数据时，严格按以下格式输出：

<function_calls>
<invoke name="sql_query">
<parameter name="query">SELECT * FROM users WHERE age > 25</parameter>
</invoke>
</function_calls>

## 示例对话

用户："查询年龄大于30的用户"
助手：我将为你查询年龄大于30的用户信息。

<function_calls>
<invoke name="sql_query">
<parameter name="query">SELECT * FROM users WHERE age > 30</parameter>
</invoke>
</function_calls>

注意：
- 必须使用工具获取数据，不能凭空回答
- SQL语句要准确，避免语法错误
- 工具调用后不要继续生成其他内容
"""

print("✅ 优化版系统提示词构建完成")


✅ 优化版系统提示词构建完成


In [7]:
# 模拟SQL查询工具
def sql_query(query):
    """
    模拟执行SQL查询
    实际项目中这里会连接真实数据库
    """
    # 模拟数据库响应
    mock_data = {
        "users": [
            {"id": 1, "name": "张三", "age": 28, "city": "北京"},
            {"id": 2, "name": "李四", "age": 32, "city": "上海"}, 
            {"id": 3, "name": "王五", "age": 25, "city": "广州"},
            {"id": 4, "name": "赵六", "age": 30, "city": "深圳"}
        ]
    }
    
    print(f"🔍 执行SQL查询: {query}")
    
    # 简单模拟查询结果
    if "age > 25" in query:
        result = [user for user in mock_data["users"] if user["age"] > 25]
    elif "COUNT" in query.upper():
        result = [{"count": len(mock_data["users"])}]
    else:
        result = mock_data["users"]
    
    return result

# 测试工具函数
test_result = sql_query("SELECT * FROM users WHERE age > 25")
print("📋 查询结果:", test_result)


🔍 执行SQL查询: SELECT * FROM users WHERE age > 25
📋 查询结果: [{'id': 1, 'name': '张三', 'age': 28, 'city': '北京'}, {'id': 2, 'name': '李四', 'age': 32, 'city': '上海'}, {'id': 4, 'name': '赵六', 'age': 30, 'city': '深圳'}]


In [8]:
# 工具调用解析函数
def parse_function_calls(response):
    """
    解析AI响应中的工具调用
    """
    import re
    
    # 查找function_calls标签
    pattern = r'<function_calls>(.*?)</function_calls>'
    matches = re.findall(pattern, response, re.DOTALL)
    
    if not matches:
        return None
        
    # 解析工具调用内容
    call_content = matches[0]
    
    # 提取工具名称
    name_pattern = r'<invoke name="([^"]+)">'
    name_match = re.search(name_pattern, call_content)
    if not name_match:
        return None
        
    tool_name = name_match.group(1)
    
    # 提取参数
    param_pattern = r'<parameter name="([^"]+)">([^<]+)</parameter>'
    param_matches = re.findall(param_pattern, call_content)
    
    parameters = {}
    for param_name, param_value in param_matches:
        parameters[param_name] = param_value.strip()
    
    return {
        "name": tool_name,
        "parameters": parameters
    }

print("✅ 工具调用解析函数定义完成")


✅ 工具调用解析函数定义完成


### 2. 完整的工具使用流程

现在我们来演示完整的工具使用过程：


In [9]:
# 优化版工具使用示例
def run_tool_example():
    """
    演示完整的工具使用流程 - 优化版
    """
    print("=" * 60)
    print("🚀 开始工具使用演示（优化版）")
    print("=" * 60)
    
    # 用户请求
    user_query = "帮我查询一下年龄大于25岁的用户有哪些？"
    print(f"👤 用户提问: {user_query}")
    print()
    
    # 第一轮：AI调用工具
    print("🤖 AI第一轮响应:")
    response_1 = get_completion(user_query, system_prompt)
    print(f"原始响应: {response_1}")
    print()
    
    # 解析工具调用
    function_call = parse_function_calls(response_1)
    
    if function_call:
        print("🔧 ✅ 检测到工具调用:")
        print(f"   📋 工具名: {function_call['name']}")
        print(f"   📝 参数: {function_call['parameters']}")
        print()
        
        # 执行工具
        if function_call['name'] == 'sql_query':
            query = function_call['parameters']['query']
            result = sql_query(query)
            print(f"✅ 工具执行结果: {result}")
            print()
            
            # 构建包含结果的新提示
            follow_up_prompt = f"""
之前的查询：{user_query}

工具调用结果：
查询语句：{query}
查询结果：{result}

请根据查询结果给出详细的分析和回答。
"""
            
            # 第二轮：AI分析结果
            print("🤖 AI最终分析:")
            final_response = get_completion(follow_up_prompt, "")
            print(final_response)
    else:
        print("❌ 未检测到工具调用！")
        print("🔍 这可能是因为：")
        print("   1. 系统提示词不够明确")
        print("   2. AI模型没有按预期格式输出")
        print("   3. 解析函数有问题")
        print()
        
        # 备用方案：直接使用更强的提示
        print("🔄 尝试使用更明确的提示...")
        stronger_prompt = f"""
用户问题：{user_query}

请严格按照以下格式调用工具查询数据库：

<function_calls>
<invoke name="sql_query">
<parameter name="query">SELECT * FROM users WHERE age > 25</parameter>
</invoke>
</function_calls>

必须使用这个格式，不要添加其他内容。
"""
        
        response_2 = get_completion(stronger_prompt, system_prompt)
        print(f"强化提示响应: {response_2}")
        
        # 再次尝试解析
        function_call_2 = parse_function_calls(response_2)
        if function_call_2:
            print("🎉 第二次尝试成功检测到工具调用！")
            query = function_call_2['parameters']['query']
            result = sql_query(query)
            print(f"✅ 工具执行结果: {result}")
        else:
            print("❌ 第二次尝试仍然失败")
    
    print()
    print("=" * 60)
    print("✅ 工具使用演示完成")
    print("=" * 60)

# 运行优化版示例
run_tool_example()


🚀 开始工具使用演示（优化版）
👤 用户提问: 帮我查询一下年龄大于25岁的用户有哪些？

🤖 AI第一轮响应:
原始响应: 我将为你查询年龄大于25岁的用户信息。

<function_calls>
<invoke name="sql_query">
<parameter name="query">SELECT * FROM users WHERE age > 25</parameter>
</invoke>
</function_calls>

🔧 ✅ 检测到工具调用:
   📋 工具名: sql_query
   📝 参数: {'query': 'SELECT * FROM users WHERE age > 25'}

🔍 执行SQL查询: SELECT * FROM users WHERE age > 25
✅ 工具执行结果: [{'id': 1, 'name': '张三', 'age': 28, 'city': '北京'}, {'id': 2, 'name': '李四', 'age': 32, 'city': '上海'}, {'id': 4, 'name': '赵六', 'age': 30, 'city': '深圳'}]

🤖 AI最终分析:
根据查询结果，我们找到了3位年龄大于25岁的用户，以下是详细分析：

1. 用户列表：
   - 用户1: 
     * ID: 1
     * 姓名: 张三
     * 年龄: 28岁
     * 城市: 北京
   - 用户2: 
     * ID: 2
     * 姓名: 李四
     * 年龄: 32岁
     * 城市: 上海
   - 用户3: 
     * ID: 4
     * 姓名: 赵六
     * 年龄: 30岁
     * 城市: 深圳

2. 年龄分布：
   - 最年长的用户是李四（32岁）
   - 最年轻的用户是张三（28岁）
   - 平均年龄: (28+32+30)/3 = 30岁

3. 地域分布：
   - 北京: 1人
   - 上海: 1人
   - 深圳: 1人
   - 用户均匀分布在三个一线城市

4. 数据完整性：
   - 查询结果中ID为3的用户未出现，可能该用户年龄≤25岁或不存在
   - 所有返回记录都完整包含id、name、ag

### 3. 关键要点总结

**系统提示词要点：**
- 📝 清晰描述工具功能和参数
- 🎯 提供具体调用格式示例  
- ⚠️ 说明使用注意事项

**控制逻辑要点：**
- 🔍 解析AI的工具调用请求
- ⚡ 执行对应的工具函数
- 🔄 返回结果给AI继续对话

**最佳实践：**
- 使用标准 `<function_calls>` 格式
- 添加清晰的错误处理
- 提供完整的使用文档


### 3. 工具使用的关键要点

**系统提示词设计：**
- 📝 清晰描述每个工具的功能和参数
- 🎯 给出具体的调用格式示例
- ⚠️ 说明注意事项和限制

**控制逻辑实现：**
- 🔍 准确解析AI的工具调用请求
- ⚡ 快速执行对应的工具函数  
- 🔄 将结果返回给AI继续对话

**最佳实践：**
- 使用标准的 `<function_calls>` 格式
- 工具函数要有清晰的错误处理
- 系统提示词要包含完整的使用说明
